<a href="https://colab.research.google.com/github/arlettecaba-15/Arlette-Caballero/blob/main/RNN/Ejercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ref: https://www.banxico.org.mx/SieInternet/consultarDirectorioInternetAction.do?accion=consultarCuadro&idCuadro=CP151&sector=8&locale=es


In [ ]:
import pandas as pd

df = pd.read_excel('/content/Consulta_20241113-093525023.xlsx')
df.head(20)

Importa pandas para manejar datos.

Carga un archivo Excel llamado Consulta_20241113-093525023.xlsx desde la ruta /content/.

Muestra las primeras 20 filas de la tabla (como vista previa).

In [ ]:
# Seleccionar la fila 8 como encabezado (índice 7)
new_header = df.iloc[8]
new_header

Toma la fila 8 del DataFrame.


La guarda como encabezado nuevo (new_header).



In [ ]:
# Crear un nuevo DataFrame desde la fila 17 en adelante
df = df[17:]
df

Elimina las primeras 17 filas del archivo.

Se queda con el resto y lo guarda en df.

In [ ]:
# Asignar el nuevo encabezado
df.columns = new_header
df.head()

Asigna new_header como nombres de columnas del DataFrame.

Muestra las primeras 5 filas con los nuevos encabezados.

In [ ]:
# Reiniciar el índice para que comience desde 0 en el nuevo DataFrame
df.reset_index(drop=True, inplace=True)
df.head()

Reinicia los índices del DataFrame (0, 1, 2…) y descarta el anterior.

Muestra las primeras 5 filas.

In [ ]:
# Convertir la columna 'fecha' a tipo datetime
df['Fecha'] = pd.to_datetime(df['Título'])
df.head()

Convierte la columna Título a formato fecha y la guarda en Fecha.

Muestra las primeras 5 filas.

In [ ]:
df.info()

Muestra información del DataFrame: columnas, tipo de datos, y valores no nulos.

In [ ]:
# Filtrar por el año 2023
_df = df[(df['Fecha'] >= '01-01-2014') & (df['Fecha'] <= '12-31-2024')]
_df.head()

Filtra las filas donde la fecha está entre 01-01-2014 y 31-12-2024.

Muestra las primeras 5 filas del resultado.

In [ ]:
_df.tail()

Muestra las últimas 5 filas del DataFrame filtrado _df.

In [ ]:
import matplotlib.pyplot as plt

# dates = pd.to_datetime(_df["Fecha"]).apply(lambda x: x.strftime('%Y-%m-%d')).tolist()
dates = _df["Fecha"].tolist()

# Grafica el "Índice Nacional de Precios al consumidor, variación anual"
plt.figure(figsize=(16, 10))
plt.plot(dates,_df["Índice Nacional de Precios al consumidor, variación anual"], color='blue', label="Variación anual")
plt.title("Índice Nacional de Precios al Consumidor - variación anual")
plt.xlabel("Tiempo")
plt.ylabel("Variación anual")
plt.xticks(rotation=90)
plt.legend()
plt.show()


Convierte las fechas a lista.

Grafica la variación anual del IPC a lo largo del tiempo.

Configura título, etiquetas, leyenda y rotación de fechas.

In [ ]:
X = _df.drop(columns=["Índice Nacional de Precios al consumidor, variación anual", "Título", "Fecha"])
y = _df["Índice Nacional de Precios al consumidor, variación anual"].values


X contiene las columnas de predictores (sin la columna objetivo, título ni fecha).


y contiene los valores del IPC que queremos predecir.



In [ ]:
X.shape

Muestra la forma de X: número de filas y columnas.

In [ ]:
y.shape

Muestra la forma de y: cantidad de valores (filas) del objetivo.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.reshape(-1, 1))

Normaliza X y y entre 0 y 1 usando MinMaxScaler.

Prepara los datos para modelos que requieren escalado.

In [ ]:
import numpy as np

window_size = 12
X_rnn = []
y_rnn = []
# Construye secuencias temporales
for i in range(window_size, len(X_scaled)):
    X_rnn.append(X_scaled[i - window_size:i])
    y_rnn.append(y_scaled[i])

X_rnn, y_rnn = np.array(X_rnn), np.array(y_rnn)

Crea secuencias de 12 pasos para entrenar la RNN.

X_rnn son las secuencias de entrada, y_rnn los valores objetivo.

In [ ]:
split = int(len(X) * 0.8)
X_train, y_train = X_rnn[:split], y_rnn[:split]
X_test, y_test = X_rnn[split:], y_rnn[split:]

Divide los datos en 80% entrenamiento y 20% prueba para la RNN.

X_train, y_train son para entrenar, X_test, y_test para evaluar.

In [ ]:
X_train.shape

Muestra la forma de X_train: cantidad de secuencias, tamaño de ventana y número de características.



```
# Tiene formato de código
```



In [ ]:
y_test.shape

Muestra la forma de y_train: cantidad de muestras de entrenamiento y dimensiones del objetivo.

In [ ]:
window_size

Muestra el tamaño de la ventana usado para las secuencias temporales: 12 pasos.

In [ ]:
X_train.shape[2]

Muestra el número de características (columnas) en cada paso de la secuencia de X_train.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, SimpleRNN, GRU

model = Sequential([
  SimpleRNN(units=60, return_sequences=True, input_shape=(window_size, X_train.shape[2])),
  SimpleRNN(units=30, return_sequences=True),
  SimpleRNN(units=15, return_sequences=False),
  Dense(units=1)
])


Crea un modelo RNN secuencial con 3 capas SimpleRNN y 1 capa Dense.

Cada RNN procesa secuencias de 12 pasos con múltiples características para predecir un valor.

In [ ]:
from tensorflow.keras.optimizers import Adam

# Compilar el modelo
learning_rate = 0.001
adam_optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=adam_optimizer, loss='mean_squared_error')

Configura el optimizador Adam con tasa de aprendizaje 0.001.

Compila el modelo RNN usando error cuadrático medio como función de pérdida.

In [ ]:
# Entrenar el modelo
model.fit(X_train, y_train, batch_size=1, epochs=10, validation_data=(X_test, y_test))

Entrena la RNN con X_train y y_train por 10 épocas.

Usa batch de 1 y valida con X_test, y_test.

In [ ]:
# Predice en el conjunto de prueba y desescala los datos
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_original = scaler.inverse_transform(y_test)

# Muestra algunas predicciones y valores reales
print("Predicciones:", predictions.flatten()[:5])
print("Valores reales:", y_test_original.flatten()[:5])


Predice con la RNN sobre X_test.

Desescala predicciones y valores reales a su rango original.

Muestra los primeros 5 valores predichos y reales.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Calcula el MAE
mae = mean_absolute_error(y_test, predictions)
print(f"Error Absoluto Medio (MAE): {mae}")

# Calcula el MSE
mse = mean_squared_error(y_test, predictions)
print(f"Error Cuadrático Medio (MSE): {mse}")

# Calcula el RMSE
rmse = np.sqrt(mse)
print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse}")

# Calcula el Coeficiente de Determinación (R²)
r2 = r2_score(y_test, predictions)
print(f"Coeficiente de Determinación (R²): {r2}")


Calcula métricas de desempeño de la RNN:

MAE (error absoluto medio)

MSE (error cuadrático medio)

RMSE (raíz del MSE)

R² (ajuste del modelo)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Gráfica de los valores reales y las predicciones del conjunto de prueba
valid = _df[split:]
valid = valid.reset_index(drop=True)
valid['Predictions'] = np.nan
valid.loc[window_size:, 'Predictions'] = predictions

dates_valid = pd.to_datetime(valid['Fecha']).apply(lambda x: x.strftime('%Y-%m-%d')).tolist()

plt.figure(figsize=(16, 8))
plt.title('Modelo RNN para Predicción de la inflación')
plt.xlabel('Fecha')
plt.ylabel('Inflacion')
plt.plot(dates_valid, valid[["Índice Nacional de Precios al consumidor, variación anual", 'Predictions']])
plt.legend(['Valor Real', 'Predicciones'], loc='lower right')
plt.xticks(rotation=90)
plt.show()

Crea un DataFrame valid con las filas de prueba y agrega columna de predicciones.


Grafica valores reales vs predicciones de inflación a lo largo del tiempo.

